In [1]:
%matplotlib inline
import sys, re
import numpy as np
import pandas as pd
import csv
import nltk
from bs4 import BeautifulSoup
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from matplotlib import pyplot as plt
pd.options.display.max_rows = 4000

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/visualboardwalk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
pd.options.display.max_rows = 4000
survey_data = pd.read_csv(r'/Volumes/GoogleDrive/Shared drives/MediSprout/vishaal/MediSprout-notebooks/NLP Analysis/survey_data.csv')
survey_data

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/GoogleDrive/Shared drives/MediSprout/vishaal/MediSprout-notebooks/NLP Analysis/survey_data.csv'

In [ ]:
survey_data.loc[survey_data['rating'] == '1024 x 1024']

In [ ]:
survey_data['rating'].dropna().astype(int)

In [ ]:
survey_data.loc[survey_data['rating'] != int]

In [ ]:
rating_prop = survey_data['rating'].value_counts(normalize=True) * 100
rating_prop

In [ ]:
rating_count = survey_data['rating'].value_counts()
rating_count

In [ ]:
rating_values = survey_data['rating'].value_counts().values
rating_values

In [ ]:
rating_df = pd.DataFrame(survey_data['rating'].unique())
rating_df

In [ ]:
survey_data.loc[survey_data['is_satisfied'] == '12.4.8']

In [ ]:
survey_data['is_satisfied'].astype(int)

In [ ]:
#survey_data['is_satisfied'] = survey_data['is_satisfied'].drop(['12.4.8'])
satisfaction_prop = survey_data['is_satisfied'].value_counts(normalize=True) * 100

In [ ]:
satisfaction_count = survey_data['is_satisfied'].value_counts()
satisfaction_count

In [ ]:
satisfaction_values = survey_data['is_satisfied'].value_counts().values
satisfaction_values

In [ ]:
satisfied_df = pd.DataFrame(survey_data['is_satisfied'].unique())

In [ ]:
survey_data['rating'].value_counts()

In [ ]:
survey_data.shape

In [ ]:
#survey_data.replace(np.nan, 'Null',regex=True)

In [ ]:
pd.crosstab(survey_data['is_satisfied'].fillna('NaN'), survey_data['rating'].fillna('NaN'), margins=True, margins_name='Total')

In [ ]:
pd.pivot_table(survey_data, values = 'is_satisfied', index = 'rating', dropna=False)

In [ ]:
#satisfaction proportions per rating(1 to 5)
sat_v_rating = pd.crosstab(survey_data['is_satisfied'].fillna('NaN'), survey_data['rating'].fillna('NaN'),\
            margins=True, margins_name='Total', normalize='index')
sat_v_rating

In [ ]:
#patient vs clinician ratings
pat_v_clin_rating = pd.crosstab(survey_data['source'].fillna('NaN'), survey_data['rating'].fillna('NaN'), \
                        margins=True, margins_name='Total', normalize='index')
pat_v_clin_rating

In [ ]:
#patient vs clinician satisfaction
pat_vs_clin_satisfaction = pd.crosstab(survey_data['source'].fillna('NaN'), survey_data['is_satisfied'].fillna('NaN'), \
                        margins=True, margins_name='Total', normalize='index')
pat_vs_clin_satisfaction

In [ ]:
#mobile vs desktop
mobile_v_desktop = pd.crosstab(survey_data['platform'].fillna('NaN'), survey_data['is_satisfied'].fillna('NaN'), \
                        margins=True, margins_name='Total', normalize='index')


In [ ]:
#mobile platforms
pd.options.display.max_rows = 4000
only_mobile = survey_data[survey_data['is_mobile'] == 1]
platform = pd.crosstab(only_mobile['platform'].fillna('NaN'), only_mobile['is_mobile'].fillna('NaN'), \
                        normalize='all')
platform = pd.DataFrame(platform).reset_index()
new_mobile_df = platform.rename(columns={1: 'percentage'})
new_mobile_df
#new_df.nlargest(5, 'percentage', keep='all')

In [ ]:
desktop_platforms = pd.Series([item for item in survey_data['platform'].str.findall('^W\w*\s\w*|^M\w*\s\w*|^L[i,e]\w*\s\w*')])
desktop_platforms = desktop_platforms.dropna().str[0].head(2000)
desktop_platforms

In [ ]:
#desktop platforms
desktops = survey_data[survey_data['is_mobile'] == 0]
only_mobile = survey_data[survey_data['is_mobile'] == 1]
platform = pd.crosstab(desktop_platforms.fillna('NaN'), only_mobile['is_mobile'].fillna('NaN'), \
                        normalize='all')
platform = pd.DataFrame(platform).reset_index()
new_desktop_df = platform.rename(columns={0: 'percentage'})
new_desktop_df
#new_df.nlargest(5, 'percentage', keep='all')

In [ ]:
#mobile vs desktop platforms - needs to output both desktop and mobile platforms
platform = pd.crosstab(survey_data['platform'].fillna('NaN'), survey_data['is_mobile'].fillna('NaN'), \
                        margins=True, margins_name='Total', normalize='all')
platform

In [ ]:
#avg satisfaction per month
survey_data['created_on'] = pd.to_datetime(survey_data['created_on'], format='%Y-%m-%d %H:%M:%S.%f')
survey_data['created_on_date'] = survey_data['created_on'].dt.date
survey_data['months'] = pd.to_datetime(survey_data['created_on'], format='%Y-%m-%d %H:%M:%S.%f').dt.strftime('%Y-%m')
avg_satisfaction = pd.crosstab(survey_data['months'], survey_data['source'], values = survey_data['is_satisfied'], \
                        margins=True, margins_name='Total', normalize='index', aggfunc = 'mean')
avg_satisfaction

In [ ]:
#avg rating per month
survey_data['created_on'] = pd.to_datetime(survey_data['created_on'], format='%Y-%m-%d %H:%M:%S.%f')
survey_data['created_on_date'] = survey_data['created_on'].dt.date
survey_data['months'] = pd.to_datetime(survey_data['created_on'], format='%Y-%m-%d %H:%M:%S.%f').dt.strftime('%Y-%m')
avg_rating = pd.crosstab(survey_data['months'], survey_data['source'], values = survey_data['rating'], \
                        margins=True, margins_name='Total', normalize='index', aggfunc='mean')
avg_rating

In [ ]:
survey_data['platform'].unique()

In [ ]:
mobile_platforms = pd.Series([item for item in survey_data['platform']\
                              .str.findall('^Apple iP\w*|^samsung\sSM-\w*|^Google\s[a-zA-Z]\w*\s\w\s\w*|^LGE\s\w*|^HTC\s\w*|^asus\s\w*|^motorola\s\w')])
mobile_platforms = mobile_platforms.str[0].dropna()
mobile_platforms

In [ ]:
#apple
apple_ = pd.Series([item for item in survey_data['platform']\
                    .str.findall('^Apple iP\w*')])
apple_platforms = apple_.str[0].dropna().head(3000)
apple_platforms

In [ ]:
apple_platforms

In [ ]:
dimensions = survey_data['screen_size'].str.split('x').dropna().head(1000)
ss1 = dimensions.apply(lambda x:x[0]).astype(int)
ss2 = dimensions.apply(lambda x:x[1]).astype(int)
total_res = ss1 * ss2

In [ ]:
def classify_device_type(row):
    platform = row['platform']
    tot_res = row['total_resolution']
    
    if bool(re.match('Apple iP\w*', platform)) == True and tot_res >= 1425636.0:
        row['device_type'] = 'Tablet'
    else:
        row['device_type'] = np.where(tot_res < 547800.0, 'Phone',\
                                            np.where(tot_res > 927408.0, 'Desktop', 'Tablet'))
    return row

In [ ]:
def classify_mobile_type(row):
    platform = row['platform']
    
    if row['is_mobile'] == 1:
        if bool(re.match('Apple iP\w*', platform)) == True:
            row['mobile_type'] = 'iOS'
        else:
            row['mobile_type'] = 'Android'
    else:
        row['mobile_type'] = 'NA'
    
    return row

In [ ]:
#resolution by device type
displays_df = pd.DataFrame\
({'id': survey_data['id'], 'platform': survey_data['platform'],\
  'screen_size': survey_data['screen_size'],'ss1':ss1,'ss2':ss2, 'total_resolution': total_res,\
  'client_name': survey_data['client_name'], \
  'is_satisfied': survey_data['is_satisfied'], \
  'is_mobile': survey_data['is_mobile'], \
  'n_minute_duration': survey_data['n_minute_duration'], \
  'feedback': survey_data['feedback']}).dropna()
displays_df

In [ ]:
classify_mobile_type(displays_df.loc[277])

In [ ]:
devices = pd.DataFrame(displays_df.apply(lambda row: classify_device_type(row), axis=1))
devices = devices.apply(lambda row: classify_mobile_type(row), axis=1)
devices

In [ ]:
devices['feedback']

In [ ]:
#Desktop: Chrome v. Firefox
desktops_only = pd.DataFrame(devices[devices['device_type'] == 'Desktop']).reset_index()

desktop_client_prop = pd.crosstab(desktops_only['platform'], desktops_only['is_satisfied'], \
                        margins=True, margins_name='Total', normalize='index')
desktop_client_prop

In [ ]:
#iOS v. Android v. Desktop
mobile_only = pd.DataFrame(devices[devices['is_mobile'] == 1]).reset_index()

mobile_client_prop = pd.crosstab(mobile_only['mobile_type'], mobile_only['is_satisfied'], \
                        margins=True, margins_name='Total', normalize='all')
mobile_client_prop

In [ ]:
#visit duration v feedback (is satisfied), call describe() to look at distributions - data.groupby('is_satisfied')['n_minute_duration'].describe()
#data.groupby('rating')['n_minute_duration'].describe()
#try to figure out short, medium, and long durations - define the thresholds - by feedback given(based off rating and is_satisfied) and by duration


In [ ]:
survey_data['n_minute_duration'].describe()

In [ ]:
survey_data.groupby('is_satisfied')['n_minute_duration'].describe()

In [ ]:
survey_data.groupby('rating')['n_minute_duration'].describe()

In [ ]:
def duration_intervals(row):
    duration = devices['n_minute_duration'].astype(int)
    
    if duration <= 10.00:
        survey_data['duration_type'] = 'Short'
    elif duration >= 11.00 and duration <= 15.00:
        survey_data['duration_type'] = 'Medium'
    else:
        survey_data['duration_type'] = 'Long'

In [ ]:
devices['feedback']

In [ ]:
plat_dimensions = pd.Series(displays_df.groupby('platform')['total_resolution'].unique())
plat_dimensions

In [ ]:
min_dimensions = pd.Series(displays_df.groupby('platform')['total_resolution'].min())
min_dimensions.sort_values()

In [ ]:
max_dimensions = pd.Series(displays_df.groupby('platform')['total_resolution'].max())
max_dimensions.sort_values()

In [ ]:
client_name = pd.Series(displays_df.groupby('platform')['client_name'].max())
client_name.sort_values()

In [ ]:
is_satisfied = pd.Series(displays_df.groupby('client_name')['is_satisfied'])
is_satisfied

In [ ]:
max_min_dimensions = pd.DataFrame\
({'min_dimensions': min_dimensions, 'plat_dimensions': plat_dimensions, 'max_dimensions': max_dimensions, 'client_name': client_name,\
 'is_satisfied': is_satisfied}).dropna()
max_min_dimensions.sort_values(['max_dimensions'])

max_min_dimensions = max_min_dimensions.reset_index()
max_min_dimensions

In [ ]:
def classify_device_type(row):
    platform = row['platform']
    max_res = row['max_dimensions']
    
    if bool(re.match('Apple iP\w*', platform)) == True and max_res >= 1425636.0:
        row['device_type'] = 'Tablet'
    else:
        row['device_type'] = np.where(row['max_dimensions'] < 547800.0, 'Phone',\
                                            np.where(row['max_dimensions'] >= 1296000.0, 'Desktop', 'Tablet'))
    return row

In [ ]:
classify_device_type(max_min_dimensions.loc[16])

In [ ]:
devices = pd.DataFrame(max_min_dimensions.apply(lambda row: classify_device_type(row), axis=1))
devices

In [ ]:
max_min_dimensions.loc[16]

In [ ]:
browsers = pd.Series(survey_data['client_name']).str.findall('^Chrome|^Firefox')
browsers = browsers.dropna().str[0]
browsers

In [ ]:
desktop_type = pd.DataFrame(devices.loc[devices['device_type'] == 'Desktop']).reset_index()
desktop_type

In [ ]:
client_prop = pd.crosstab(devices['client_name'], desktop_type['is_satisfied'], \
                        margins=True, margins_name='Total', normalize='index')
client_prop